In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib_venn import venn2

import shap
from optuna.integration import _lightgbm_tuner as lgb_tuner
import optuna

from catboost import CatBoost
from catboost import Pool
from catboost import cv
import category_encoders as ce
from tqdm import tqdm

import lightgbm as lgb
import xgboost as xgb
import joblib

import os
import logging
import datetime

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

from catboost import CatBoostRegressor

In [ ]:
df_train = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/train.csv")
df_test = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/test.csv")
submission = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv")

In [ ]:
print(df_train.info())
print("-"*30)
print(df_test.info())

# EDA

In [ ]:
fig = plt.figure(figsize=(20,20))

for i in range(1,15):
    plt.subplot(4,4,i)
    sns.distplot(df_train["cont"+str(i)], label='train')
    sns.distplot(df_test["cont"+str(i)], label='test')
    plt.legend()
    plt.grid()

In [ ]:
colname = list()

fig = plt.figure(figsize=(20,20))

for i in range(1,15):
    colname.append('cont'+str(i))

df_train[colname].plot(figsize=(10,5), kind='box', rot=90, fontsize=12, title='train', grid='gainsboro')
df_test[colname].plot(figsize=(10,5), kind='box', rot=90, fontsize=12, title='test', grid='gainsboro')
plt.show()

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 10))
ax.set_title("Correlation Matrix", fontsize=16)
sns.heatmap(df_train[df_train.columns[df_train.columns != 'id']].corr(), vmin=-1, vmax=1, cmap='jet', annot=True)

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(14) 
    tick.label.set_rotation(90) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(14)
    tick.label.set_rotation(0) 
plt.show()

In [ ]:
sns.distplot(df_train["target"])

In [ ]:
y = df_train["target"]
X = df_train.drop(["target","id"], axis=1)

# LightGBM_Optuna

In [ ]:
from sklearn.metrics import mean_squared_log_error
from contextlib import contextmanager
from time import time

fold_num = 10
EARLY_STOPPING_ROUNDS = 10
VERBOSE_EVAL = 10000
LGB_ROUND_NUM = 10000
objective = 'regression'
metric = 'rmse'

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': objective,
    'metric': metric,
    'verbosity': -1,
    "seed": 42,
}


@contextmanager
def timer(logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None):
    if prefix: format_str = str(prefix) + format_str
    if suffix: format_str = format_str + str(suffix)
    start = time()
    yield
    d = time() - start
    out_str = format_str.format(d)
    if logger:
        logger.info(out_str)
    else:
        print(out_str)

def fit_lgbm(X, y, cv, params: dict=None, verbose: int=50):
    metric_func = mean_squared_error

    if params is None:
        params = {}

    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]
        
        lgb_train = lgb.Dataset(x_train, y_train)
        lgb_valid = lgb.Dataset(x_valid, y_valid)

        with timer(prefix='fit fold={} '.format(i + 1)):
            lgb_model = lgb_tuner.train(params, lgb_train,
                          num_boost_round=LGB_ROUND_NUM,
                          valid_names=["train", "valid"],
                          valid_sets=[lgb_train, lgb_valid],
                          early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                          verbose_eval=VERBOSE_EVAL)

        pred_i = lgb_model.predict(x_valid)

        oof_pred[idx_valid] = pred_i
        models.append(lgb_model)

        print(f'Fold {i} RMSE: {metric_func(y_valid, pred_i) ** .5:.4f}')

    score = metric_func(y, oof_pred) ** .5
    print('FINISHED | Whole RMSE: {:.4f}'.format(score))
    return oof_pred, models

In [ ]:
from sklearn.model_selection import KFold, GroupKFold



fold = KFold(n_splits=5, shuffle=True, random_state=71)
cv = list(fold.split(X, y)) # もともとが generator なため明示的に list に変換する

oof, models = fit_lgbm(X.values, y, cv, params=params)

In [ ]:
def visualize_importance(models, feat_train_df):
    """
    args:
        models:
            List of lightGBM models
        feat_train_df:
            DataFrame
    """
    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importance()
        _df['column'] = feat_train_df.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], axis=0, ignore_index=True)

    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(max(6, len(order) * .4), 7))
    sns.boxenplot(data=feature_importance_df, x='column', y='feature_importance', order=order, ax=ax, palette='viridis')
    ax.tick_params(axis='x', rotation=90)
    ax.grid()
    fig.tight_layout()
    return fig, ax

In [ ]:
fig, ax = visualize_importance(models, X)

# XGB

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

def opt(trial):
    # params
    n_estimators = trial.suggest_int('n_estimators', 0, 1000)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    learning_rate = trial.suggest_discrete_uniform('learning_rate', 0.01,0.1,0.01)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 20)
    subsample = trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1)
    colsample_bytree = trial.suggest_discrete_uniform('colsample_bytree', 0.5, 0.9, 0.1)
    # XGBostのインスタンス生成
    xgboost_tuna = xgb.XGBRegressor(
        random_state=42,
        n_estimators = n_estimators,
        max_depth = max_depth,
        min_child_weight = min_child_weight,
        subsample = subsample,
        colsample_bytree = colsample_bytree,
    )
    # train
    xgboost_tuna.fit(X_train,y_train)
    # pred
    tuna_pred_test = xgboost_tuna.predict(X_test)
    # RMSE
    return mean_squared_error(y_test, tuna_pred_test)

In [ ]:
#study = optuna.create_study()
#study.optimize(opt, n_trials=50)

In [ ]:
#print(study.best_trial)

In [ ]:
def fit_xgb(X, y, cv, params: dict=None, verbose: int=50):
    metric_func = mean_squared_error
    if params is None:
        params = {}

    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]
        
        model_xgb = xgb.XGBRegressor(**params)

        with timer(prefix='fit fold={} '.format(i + 1)):
            model_xgb.fit(x_train, y_train, eval_set=[(x_valid, y_valid)],verbose=-1)
            
        #print(model_xgb.best_score())
        
        pred_i = model_xgb.predict(x_valid)

        oof_pred[idx_valid] = pred_i
        models.append(model_xgb)

        print(f'Fold {i} RMSE: {metric_func(y_valid, pred_i) ** .5:.4f}')

    score = metric_func(y, oof_pred) ** .5
    print('FINISHED | Whole RMSE: {:.4f}'.format(score))
    return oof_pred, models

In [ ]:
"""
FrozenTrial(number=45, value=0.4896166448243632, datetime_start=datetime.datetime(2021, 1, 12, 15, 44, 13, 759937),
datetime_complete=datetime.datetime(2021, 1, 12, 15, 44, 48, 234726), params={'n_estimators': 208, 'max_depth': 4,
'min_child_weight': 13, 'subsample': 0.8, 'colsample_bytree': 0.8}, 
distributions={'n_estimators': IntUniformDistribution(high=1000, low=0, step=1),
'max_depth': IntUniformDistribution(high=20, low=1, step=1), 'min_child_weight':
IntUniformDistribution(high=20, low=1, step=1),
'subsample': DiscreteUniformDistribution(high=0.9, low=0.5, q=0.1), 
'colsample_bytree': DiscreteUniformDistribution(high=0.9, low=0.5, q=0.1)},
user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=45, state=TrialState.COMPLETE)
"""

params_xgb = {'n_estimators': 208, 
              'max_depth': 4, 
              'learning_rate':0.08,
              'min_child_weight': 13,
              'subsample': 0.8, 
              'colsample_bytree': 0.8}

oof_xgb, models_xgb = fit_xgb(X.values, y, cv, params=params_xgb)

# CatBoost

In [ ]:
# optuna

def opt_cb(trial):
    # params
    params = {
        'iterations' : trial.suggest_int('iterations', 50, 300),                         
        'depth' : trial.suggest_int('depth', 4, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),               
        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), 
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait' :trial.suggest_int('od_wait', 10, 50)
    }
    
    train_pool = Pool(X_train, y_train)
    test_pool = Pool(X_test, y_test)
    
    catboost_tuna = CatBoostRegressor(**params)
    # train
    catboost_tuna.fit(train_pool)
    # pred
    tuna_pred_test = catboost_tuna.predict(test_pool)
    pred_labels = np.rint(tuna_pred_test)
    # RMSE
    return mean_squared_error(y_test, pred_labels)

In [ ]:
#study2 = optuna.create_study()
#study2.optimize(opt_cb, n_trials=50)

In [ ]:
#print(study.best_trial)

In [ ]:
def fit_cb(X, y, cv, params: dict=None, verbose: int=50):
    metric_func = mean_squared_error
    if params is None:
        params = {}

    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]
        
        train_pool = Pool(x_train, label = y_train)
        valid_pool = Pool(x_valid, label = y_valid)
        
        model_cb = CatBoost(params)

        with timer(prefix='fit fold={} '.format(i + 1)):
            model_cb.fit(train_pool,
              # valid_data
              eval_set = valid_pool,
              use_best_model = True,
              silent = True,
              plot = False)
            
        print(model_cb.get_best_score())
        
        pred_i = model_cb.predict(x_valid)

        oof_pred[idx_valid] = pred_i
        models.append(model_cb)

        print(f'Fold {i} RMSE: {metric_func(y_valid, pred_i) ** .5:.4f}')

    score = metric_func(y, oof_pred) ** .5
    print('FINISHED | Whole RMSE: {:.4f}'.format(score))
    return oof_pred, models

In [ ]:
params_cb = {
    'loss_function': 'RMSE',
    'max_depth': 3, 
    'learning_rate': 0.08, 
    'subsample': 0.8, 
    #'colsample_bytree': 0.7,
    'num_boost_round': 1000,
    'early_stopping_rounds': 100,
}

oof_cb, models_cb = fit_cb(X.values, y, cv, params=params_cb)

# Predict

In [ ]:
df_test = df_test.drop("id",axis=1)

In [ ]:
pred_lgb = np.array([model.predict(df_test.values) for model in models])
pred_lgb = np.mean(pred_lgb, axis=0)
pred_lgb = np.where(pred_lgb < 0, 0, pred_lgb)

pred_xgb = np.array([model.predict(df_test.values) for model in models_xgb])
pred_xgb = np.mean(pred_xgb, axis=0)
pred_xgb = np.where(pred_xgb < 0, 0, pred_xgb)

pred_cb = np.array([model.predict(df_test.values) for model in models_cb])
pred_cb = np.mean(pred_cb, axis=0)
pred_cb = np.where(pred_cb < 0, 0, pred_cb)

tmp_sub = pd.DataFrame({"lgb":pred_lgb,
                        "xgb":pred_xgb,
                         "cb":pred_cb})
tmp_sub["pred"] = tmp_sub.mean(axis="columns")

In [ ]:
submission["target"] = tmp_sub["pred"].copy()

In [ ]:
submission.to_csv("submission.csv", index=False)

In [ ]:
oof_df = pd.DataFrame({"lgb":oof,
                        "xgb":oof_xgb,
                         "cb":oof_cb})
oof_df["pred"] = oof_df.mean(axis="columns")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
sns.distplot(oof_df["pred"], label='Test Predict')
sns.distplot(submission["target"], label='Out Of Fold')
ax.legend()
ax.grid()